In [ ]:
import os

import numpy as np
import tensorflow as tf

import config, music_model, utils

ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

conf = config.Config("single_instruments_type", ROOT_PATH)

In [ ]:
USE_ONE_GPU = True            # or False if another GPU is available
USE_SMALL_GENRE_SET = True    # or False if we want to use the dataset with the full genre subset list
DATASET_NAME = 'tf_data7dict' # or whatever

In [ ]:
if USE_SMALL_GENRE_SET:
    conf.accepted_subgenres = ['folk', 'nes', 'maestro']
# If we need to use only the first GPU
if USE_ONE_GPU:
    conf.GPUS = tf.config.experimental.list_physical_devices('GPU')[0]
    conf.BATCH_SIZE = 4
    conf.GLOBAL_BATCH_SIZE = conf.BATCH_SIZE
    conf.num_devices = 1

In [ ]:
if conf.num_devices > 1:
    print("Using multiple GPUs with Mirrored Strategy")
    with conf.training_strategy.scope():
        model = music_model.create_model(num_genres=len(conf.accepted_subgenres), use_masking_layers=True, training=False)
else:
    print("Using single GPU/CPU device")
    model = music_model.create_model(num_genres=len(conf.accepted_subgenres), use_masking_layers=True, training=False)

In [ ]:
song = np.zeros(conf.GENERATION_BATCH_SIZE, conf.SEQ_LEN-1, 11)

# random choice for style, can be changed
genre_one_hot = map(
    utils.one_hot_encode_labels_nmf,
    np.random.choice(["folk", "nes", "maestro"], size=conf.GENERATION_BATCH_SIZE)
)

i=0
while any(song[:, i, 0]) != 7:

    predicted_song = model((song, genre_one_hot))

    song[:, i+1, :] = predicted_song[i, :]
    i += 1
